In [367]:
'''
Code to generate various confidence bands and find the best one 
by comparing the sum of the lengths of confidence intervals at 
each time step, standard deviation and the length of maximum interval.
'''

'\nCode to generate various confidence bands and find the best one \nby comparing the sum of the lengths of confidence intervals at \neach time step, standard deviation and the length of maximum interval.\n'

In [368]:
import numpy as np
from numpy.random import randint
from math import factorial
import matplotlib.pyplot as plt
from pandas import read_csv

In [369]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.callbacks import TensorBoard

In [370]:
from time import time
import itertools

In [371]:
# Generate X and Y pairs where dim of X = (1, look_back)
def create_dataset(dataset, look_back = 1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), :]
        dataX.append(a)
        dataY.append(dataset[i + look_back, :])
    return np.array(dataX), np.array(dataY)

In [372]:
# Arrange blocks of data according to permutation
def arrange_data(data, perm):
    arranged_data = []
    for i in perm:
        arranged_data.append(data[i])
    return arranged_data

In [373]:
def bootstrap_resample(data, blockSize):
    n = len(data) # 100
    numInterval = n//blockSize # 20
    data = data[: numInterval*blockSize]    
    resampled_data = []
    for i in range(numInterval):
        x = data[i*blockSize: (i+1)*blockSize]
        indx = randint(0, blockSize, size = blockSize)
        y = [x[j] for j in indx]
        resampled_data.append(y)
    return resampled_data

In [374]:
fields = ['open', 'close']
dataframe = read_csv('GOOGL_data.csv', skipinitialspace = True, squeeze = True, usecols = fields)

print(dataframe.head())
data = np.array(dataframe)
print(data.shape)

       open     close
0  390.4551  393.0777
1  389.5892  391.6012
2  391.2659  390.7403
3  390.4551  391.8214
4  390.2549  394.3039
(1259, 2)


In [375]:
# resample = bootstrap_resample(data, 5)

# resample

# resample[1]

# arrange = arrange_data(resample, [1, 0])

# arrange[0]

In [376]:
# Scale between (0, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)
print(data[:5])

[[0.00683718 0.01210825]
 [0.0057589  0.01027231]
 [0.00784685 0.00920184]
 [0.00683718 0.01054612]
 [0.00658788 0.01363296]]


In [377]:
split = 0.72
trainSize = int(len(data)*split)
testSize = len(data)-trainSize
print(trainSize)
print(testSize)

906
353


In [378]:
train = data[0:trainSize,:]
test = data[trainSize:len(data),:]
print(train[:5])

[[0.00683718 0.01210825]
 [0.0057589  0.01027231]
 [0.00784685 0.00920184]
 [0.00683718 0.01054612]
 [0.00658788 0.01363296]]


In [379]:
# l = block size
numInterval = 4
blockSize = len(train)//numInterval
# print(blockSize)

In [380]:
blockSize

226

In [381]:
A = np.array([[[1, 2]], [[2, 3]], [[3, 4]]])
B = np.reshape(A, (A.shape[0]*A.shape[1], A.shape[2]))
B

array([[1, 2],
       [2, 3],
       [3, 4]])

In [382]:
# Generate all permutations of the series 0, 1, 2, ... numInterval
permutations = itertools.permutations(range(numInterval))
testBand = []
count = 1

In [ ]:
totalCount = factorial(numInterval)
for perm in permutations:
    print("Count = %d/%d" % (count, totalCount))
    count += 1
    trainResample = bootstrap_resample(train, blockSize)
    trainSet = np.array(arrange_data(trainResample, perm))
#     trainSet = np.reshape(trainSet, trainSet.shape)
    trainSet = np.reshape(trainSet, (trainSet.shape[0]*trainSet.shape[1], trainSet.shape[2]))

    shuffleData = np.vstack((trainSet, test))

    lookBack = 3
    trainX, trainY = create_dataset(trainSet, lookBack)
    testX, testY = create_dataset(test, lookBack)

    # trainX = np.reshape(trainX, (trainX.shape[0], 2, trainX.shape[1]))
    # testX = np.reshape(testX, (testX.shape[0], 2, testX.shape[1]))

    units = 100
    drop = 0.2
    epoch = 5

    model = Sequential()
    model.add(LSTM(units, input_shape=(lookBack, 2)))
    model.add(Dense(2))
    model.compile(loss='mean_squared_error', optimizer='nadam')
#     Tensorboard
#     model.summary()
#     tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
#     model.fit(trainX, trainY, epochs=epoch, batch_size=1, verbose=1, callbacks=[tensorboard])
    model.fit(trainX, trainY, epochs=epoch, batch_size=1, verbose=1)

    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)

#     Inverse the scaling
    trainPredict = scaler.inverse_transform(trainPredict)
    trainY = scaler.inverse_transform(trainY)
    testPredict = scaler.inverse_transform(testPredict)
    testY = scaler.inverse_transform(testY)

#     RMSE score
    trainScore = math.sqrt(mean_squared_error(trainY, trainPredict))
    print('Train Score: %.2f RMSE' % (trainScore))
    testScore = math.sqrt(mean_squared_error(testY, testPredict))
    print('Test Score: %.2f RMSE' % (testScore))

#     Plot training data
#     trainPredictPlot = np.empty_like(shuffleData)
#     trainPredictPlot[:, :] = np.nan
#     trainPredictPlot[lookBack:len(trainPredict)+lookBack, :] = trainPredict
    
#     Plot test data
#     testPredictPlot = np.empty_like(shuffleData)
#     testPredictPlot[:, :] = np.nan
#     testPredictPlot[len(trainPredict)+(lookBack*2)+1:len(data)-1, :] = testPredict
    
#     Append results to calculate the test band
    testBand.append(testPredict)
#     plt.plot(scaler.inverse_transform(shuffleData)[:,col])
#     plt.plot(trainPredictPlot[:,col], color = 'orange')
#     plt.plot(testPredictPlot[:,col], color = 'green')

Count = 1/24
Epoch 1/5
900/900 [==============================] - 7s 8ms/step - loss: 0.0055
Epoch 2/5
900/900 [==============================] - 6s 7ms/step - loss: 0.0043
Epoch 3/5
900/900 [==============================] - 6s 6ms/step - loss: 0.0042
Epoch 4/5
900/900 [==============================] - 6s 6ms/step - loss: 0.0041
Epoch 5/5
900/900 [==============================] - 6s 6ms/step - loss: 0.0041
Train Score: 54.10 RMSE
Test Score: 57.50 RMSE
Count = 2/24
Epoch 1/5
900/900 [==============================] - 8s 9ms/step - loss: 0.0047
Epoch 2/5
900/900 [==============================] - 6s 6ms/step - loss: 0.0040
Epoch 3/5
900/900 [==============================] - 6s 6ms/step - loss: 0.0039
Epoch 4/5
900/900 [==============================] - 6s 7ms/step - loss: 0.0040
Epoch 5/5
900/900 [==============================] - 6s 7ms/step - loss: 0.0039
Train Score: 49.09 RMSE
Test Score: 27.42 RMSE
Count = 3/24
Epoch 1/5
900/900 [==============================] - 8s 9ms/step - 

900/900 [==============================] - 6s 6ms/step - loss: 0.0039
Train Score: 48.84 RMSE
Test Score: 54.32 RMSE
Count = 19/24
Epoch 1/5
900/900 [==============================] - 9s 10ms/step - loss: 0.0053
Epoch 2/5
900/900 [==============================] - 5s 6ms/step - loss: 0.0044
Epoch 3/5
900/900 [==============================] - 5s 6ms/step - loss: 0.0043
Epoch 4/5
900/900 [==============================] - 5s 6ms/step - loss: 0.0042
Epoch 5/5
900/900 [==============================] - 5s 6ms/step - loss: 0.0042
Train Score: 50.37 RMSE
Test Score: 56.06 RMSE
Count = 20/24
Epoch 1/5
900/900 [==============================] - 9s 10ms/step - loss: 0.0055
Epoch 2/5
900/900 [==============================] - 6s 6ms/step - loss: 0.0045
Epoch 3/5
900/900 [==============================] - 5s 6ms/step - loss: 0.0046
Epoch 4/5
900/900 [==============================] - 5s 6ms/step - loss: 0.0045
Epoch 5/5
900/900 [==============================] - 6s 6ms/step - loss: 0.0044
Train 

In [ ]:
model.save("./Models/ResampledBlockBootstrap/"+str(numInterval)+"_"+str(epoch)+".h5")

In [ ]:
col = 0
testPlot = np.empty_like(trainSet)
testPlot[:, :] = 0
testPlot = np.vstack((testPlot, test))
testplot = scaler.inverse_transform(testPlot)
plt.plot(testPlot[:,col], color = 'blue')
plt.title('Epoch = %d Train = %.2f Test = %.2f' % (epoch, trainScore, testScore))

In [ ]:
testBand = np.array(testBand)
testBand = np.reshape(testBand, (testBand.shape))

In [ ]:
testBand.shape

In [ ]:
print(np.mean(testBand[:,0,:], axis = 0))
print(np.std(testBand[:,0,:], axis = 0))

In [ ]:
z_alpha = 1.96
n = factorial(numInterval)

confInterval = []

# Calculate the 95% confidence interval for each time step
for i in range(testBand.shape[1]):
    X = testBand[:, i, :]
    xBar = np.mean(X, axis = 0)
    s = np.std(X, axis = 0)
    l = xBar - 1.96*s/(n**0,.5)
    r = xBar + 1.96*s/(n**0,.5)
    pair = [l, r]
    confInterval.append(pair)

In [ ]:
col = 0
offset = len(trainPredict)+(lookBack*2)+2
lower = []
upper = []

# Get upper and lower bounds
for i in range(len(confInterval)):
    lower.append(confInterval[i][0][0])
    upper.append(confInterval[i][1][0])

# XLower = np.array(range(offset+1, offset+1+testBand.shape[1]))
# XUpper = np.array(range(offset+1, offset+1+testBand.shape[1]))
# plt.plot(XLower, lower)
# plt.plot(XUpper, upper)
plt.plot(lower)
plt.plot(upper)
plt.plot(scaler.inverse_transform(test)[:, col], color = 'blue')
plt.title('NumInterval=%d Epochs=%d Train=%.2f Test=%.2f' % (numInterval, epoch, trainScore, testScore))
plt.savefig("./Plots/ResampledBlockBootstrap/"+str(numInterval)+"_"+str(epoch)+".png")
plt.show()

In [ ]:
# Sum of the lengths of the confidence intervals at each time point.
sumIntervals = np.sum(abs(np.array(upper) - np.array(lower)))

In [ ]:
sumIntervals

In [ ]:
# Length of maximum interval
maxInterval = np.max(np.array(upper) - np.array(lower))

In [ ]:
maxInterval